In [1]:
from sympy import *
import numpy as np
from IPython.display import display
from itertools import product
from action import Coarsener, Action

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
phi = IndexedBase("phi")
i = symbols("i", integer=True)
j = symbols("j", integer=True)
# N = symbols("N")
# i = Idx("i",N)
# j = Idx("j",N)
alpha = IndexedBase("alpha")
beta = IndexedBase("beta")
omega = symbols("omega")
h = symbols("h")
gamma = IndexedBase("c")

fine_action_expr = phi[i, j] * (
    h**2 * alpha[i, j] * phi[i, j]
    - beta[i + 1, j] * (phi[i + 1, j] - phi[i, j])
    - beta[i, j] * (phi[i - 1, j] - phi[i, j])
    - beta[i, j + 1] * (phi[i, j + 1] - phi[i, j])
    - beta[i, j] * (phi[i, j - 1] - phi[i, j])
)

action = Action(fine_action_expr, phi, (i, j))
action_2 = action**3

with open("generated_code.cc","w",encoding="utf8") as f:
    print(action_2.get_stencil(),file=f)

In [157]:
action_3 = action**2
display(simplify(action_3.expr))
action_3_expr = action_3.expr
table = np.empty((5,5),dtype=np.int32)
for offset_i in range(-2,3):
    for offset_j in range(-2,3):
        term = phi[i,j]*phi[i+offset_i,j+offset_j]
        coeff = expand(action_3_expr).coeff(term)
        display(term)
        display(coeff)
        #table[offset_i+2,offset_j+2] = coeff
        print("----")
        action_3_expr = simplify(action_3_expr - coeff * term)



(h**4*alpha[i, j]**2*phi[i, j] - h**2*alpha[i + 1, j]*beta[i + 1, j]*phi[i + 1, j] - h**2*alpha[i - 1, j]*beta[i, j]*phi[i - 1, j] - h**2*alpha[i, j + 1]*beta[i, j + 1]*phi[i, j + 1] - h**2*alpha[i, j - 1]*beta[i, j]*phi[i, j - 1] - h**2*alpha[i, j]*beta[i + 1, j]*phi[i + 1, j] + 2*h**2*alpha[i, j]*beta[i + 1, j]*phi[i, j] - h**2*alpha[i, j]*beta[i, j + 1]*phi[i, j + 1] + 2*h**2*alpha[i, j]*beta[i, j + 1]*phi[i, j] - h**2*alpha[i, j]*beta[i, j]*phi[i - 1, j] - h**2*alpha[i, j]*beta[i, j]*phi[i, j - 1] + 4*h**2*alpha[i, j]*beta[i, j]*phi[i, j] + beta[i + 1, j + 1]*beta[i + 1, j]*phi[i + 1, j + 1] - beta[i + 1, j + 1]*beta[i + 1, j]*phi[i + 1, j] + beta[i + 1, j + 1]*beta[i, j + 1]*phi[i + 1, j + 1] - beta[i + 1, j + 1]*beta[i, j + 1]*phi[i, j + 1] + beta[i + 1, j - 1]*beta[i, j]*phi[i + 1, j - 1] - beta[i + 1, j - 1]*beta[i, j]*phi[i, j - 1] + beta[i + 1, j]**2*phi[i + 1, j - 1] - 3*beta[i + 1, j]**2*phi[i + 1, j] + 2*beta[i + 1, j]**2*phi[i, j] - beta[i + 1, j]*beta[i + 2, j]*phi[i + 1

phi[i - 2, j - 2]*phi[i, j]

0

----


phi[i - 2, j - 1]*phi[i, j]

...

----


phi[i - 2, j + 2]*phi[i, j]

0

----


KeyboardInterrupt: 

In [43]:
print(ccode(simplify(action_3.expr)))

IndexException: 
Range is not defined for all indices in: alpha[i + 1, j]

In [19]:
for offset_i in range(5):
    s = ""
    for offset_j in range(5):        
        s += "{:4d} ".format(table[offset_i,offset_j])
    print (s)
    

   0    0    1    0    0 
   0    2   -8    2    0 
   1   -8   20   -8    1 
   0    2   -8    2    0 
   0    0    1    0    0 


In [48]:
phi = IndexedBase("phi")
Phi = IndexedBase("Phi")
psi = IndexedBase("psi")
alpha = IndexedBase("a")
beta_x = IndexedBase("b^(x)")
beta_y = IndexedBase("b^(y)")
gamma = IndexedBase("c")
i = symbols("i", integer=True)
j = symbols("j", integer=True)

fine_action = (
    alpha[i, j] * phi[i, j] ** 2
    + beta_x[i, j] * phi[i, j] * phi[i + 1, j]
    + beta_y[i, j] * phi[i, j] * phi[i, j + 1]
    + gamma[i, j] * phi[i, j]
)

coarsener = Coarsener(phi, Phi, psi, [i, j])

coarse_action = coarsener.apply(fine_action)
for term in [
    Phi[i, j] ** 2,
    Phi[i, j] * Phi[i + 1, j],
    Phi[i, j] * Phi[i, j + 1],
    Phi[i, j],
]:
    coeff = coarse_action.coeff(term)
    display(term)
    display(coeff)
    print("----")
    coarse_action = simplify(coarse_action - coeff * term)
display(coarse_action)


Phi[i, j]**2

a[2*i + 1, 2*j + 1] + a[2*i + 1, 2*j] + a[2*i, 2*j + 1] + a[2*i, 2*j] + b^(x)[2*i, 2*j + 1] + b^(x)[2*i, 2*j] + b^(y)[2*i + 1, 2*j] + b^(y)[2*i, 2*j]

----


Phi[i + 1, j]*Phi[i, j]

b^(x)[2*i + 1, 2*j + 1] + b^(x)[2*i + 1, 2*j]

...

----


Phi[i, j]

2*a[2*i + 1, 2*j + 1]*psi[2*i + 1, 2*j + 1] + 2*a[2*i + 1, 2*j]*psi[2*i + 1, 2*j] + 2*a[2*i, 2*j + 1]*psi[2*i, 2*j + 1] + 2*a[2*i, 2*j]*psi[2*i, 2*j] + b^(x)[2*i + 1, 2*j + 1]*psi[2*i + 2, 2*j + 1] + b^(x)[2*i + 1, 2*j]*psi[2*i + 2, 2*j] + b^(x)[2*i - 1, 2*j + 1]*psi[2*i - 1, 2*j + 1] + b^(x)[2*i - 1, 2*j]*psi[2*i - 1, 2*j] + b^(x)[2*i, 2*j + 1]*psi[2*i + 1, 2*j + 1] + b^(x)[2*i, 2*j + 1]*psi[2*i, 2*j + 1] + b^(x)[2*i, 2*j]*psi[2*i + 1, 2*j] + b^(x)[2*i, 2*j]*psi[2*i, 2*j] + b^(y)[2*i + 1, 2*j + 1]*psi[2*i + 1, 2*j + 2] + b^(y)[2*i + 1, 2*j - 1]*psi[2*i + 1, 2*j - 1] + b^(y)[2*i + 1, 2*j]*psi[2*i + 1, 2*j + 1] + b^(y)[2*i + 1, 2*j]*psi[2*i + 1, 2*j] + b^(y)[2*i, 2*j + 1]*psi[2*i, 2*j + 2] + b^(y)[2*i, 2*j - 1]*psi[2*i, 2*j - 1] + b^(y)[2*i, 2*j]*psi[2*i, 2*j + 1] + b^(y)[2*i, 2*j]*psi[2*i, 2*j] + c[2*i + 1, 2*j + 1] + c[2*i + 1, 2*j] + c[2*i, 2*j + 1] + c[2*i, 2*j]

----


a[2*i + 1, 2*j + 1]*psi[2*i + 1, 2*j + 1]**2 + a[2*i + 1, 2*j]*psi[2*i + 1, 2*j]**2 + a[2*i, 2*j + 1]*psi[2*i, 2*j + 1]**2 + a[2*i, 2*j]*psi[2*i, 2*j]**2 + b^(x)[2*i + 1, 2*j + 1]*psi[2*i + 1, 2*j + 1]*psi[2*i + 2, 2*j + 1] + b^(x)[2*i + 1, 2*j]*psi[2*i + 1, 2*j]*psi[2*i + 2, 2*j] + b^(x)[2*i, 2*j + 1]*psi[2*i + 1, 2*j + 1]*psi[2*i, 2*j + 1] + b^(x)[2*i, 2*j]*psi[2*i + 1, 2*j]*psi[2*i, 2*j] + b^(y)[2*i + 1, 2*j + 1]*psi[2*i + 1, 2*j + 1]*psi[2*i + 1, 2*j + 2] + b^(y)[2*i + 1, 2*j]*psi[2*i + 1, 2*j + 1]*psi[2*i + 1, 2*j] + b^(y)[2*i, 2*j + 1]*psi[2*i, 2*j + 1]*psi[2*i, 2*j + 2] + b^(y)[2*i, 2*j]*psi[2*i, 2*j + 1]*psi[2*i, 2*j] + c[2*i + 1, 2*j + 1]*psi[2*i + 1, 2*j + 1] + c[2*i + 1, 2*j]*psi[2*i + 1, 2*j] + c[2*i, 2*j + 1]*psi[2*i, 2*j + 1] + c[2*i, 2*j]*psi[2*i, 2*j]